In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time
import torchvision.models as models

In [3]:
from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
from collections import namedtuple
import os
import random
import shutil
import time

In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
train_transforms = transforms.Compose([
                            transforms.Resize((224, 224)),
                            transforms.ToTensor()
                                      ])

In [6]:
valid_transforms = transforms.Compose([
                            transforms.Resize((224, 224)),
                            transforms.ToTensor()
                                      ])

In [7]:
train_data = datasets.ImageFolder(root = '/content/drive/My Drive/Retail Pulse ML Assignment Data/train',transform=train_transforms)

In [8]:
fil1 = open('/content/drive/My Drive/Retail Pulse ML Assignment Data/images_family_train.txt', 'r') 
Lin1 = fil1.readlines() 

In [9]:
c = list()

In [10]:
l = 0
temp = Lin1[0][8:]
for i in range(len(Lin1)):
    if(temp == Lin1[i][8:]):
        c.append(l)
    else:
        temp = Lin1[i][8:]
        l=l+1
        c.append(l)

In [11]:
for i in range(len(Lin1)):
    train_data.targets[i] = c[i]
print(len(train_data.targets))

3334


In [12]:
print(len(train_data.targets))

3334


In [13]:
fil2 = open('/content/drive/My Drive/Retail Pulse ML Assignment Data/images_family_val.txt', 'r') 
Lin2 = fil2.readlines() 

In [14]:
d = list()

In [15]:
l = 0
temp = Lin2[0][8:]
for i in range(len(Lin2)):
    if(temp == Lin2[i][8:]):
        d.append(l)
    else:
        temp = Lin2[i][8:]
        l=l+1
        d.append(l)

In [16]:
valid_data = datasets.ImageFolder(root = '/content/drive/My Drive/Retail Pulse ML Assignment Data/val',transform=valid_transforms)
print(len(valid_data.targets))

3333


In [17]:
BATCH_SIZE = 8
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
model = models.resnet50(pretrained=True).to(device)

for param in model.parameters():
  param.requires_grad=False
    
model.fc = nn.Sequential(
               nn.Linear(2048, 496),
               nn.ReLU(inplace=True),
               nn.Linear(496, 100)).to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,066,004 trainable parameters


In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-05)
criterion = criterion.to(device)

In [27]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [28]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [29]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [30]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [31]:
EPOCHS = 100

best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss :
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 40m 30s
	Train Loss: 4.610 | Train Acc: 0.75%
	 Val. Loss: 4.598 |  Val. Acc: 1.38%
Epoch: 02 | Epoch Time: 0m 35s
	Train Loss: 4.592 | Train Acc: 1.71%
	 Val. Loss: 4.585 |  Val. Acc: 2.19%
Epoch: 03 | Epoch Time: 0m 35s
	Train Loss: 4.573 | Train Acc: 2.94%
	 Val. Loss: 4.572 |  Val. Acc: 3.09%
Epoch: 04 | Epoch Time: 0m 35s
	Train Loss: 4.554 | Train Acc: 3.48%
	 Val. Loss: 4.555 |  Val. Acc: 3.36%
Epoch: 05 | Epoch Time: 0m 35s
	Train Loss: 4.535 | Train Acc: 5.46%
	 Val. Loss: 4.539 |  Val. Acc: 5.04%
Epoch: 06 | Epoch Time: 0m 34s
	Train Loss: 4.515 | Train Acc: 5.61%
	 Val. Loss: 4.522 |  Val. Acc: 5.67%
Epoch: 07 | Epoch Time: 0m 34s
	Train Loss: 4.494 | Train Acc: 7.01%
	 Val. Loss: 4.506 |  Val. Acc: 5.79%
Epoch: 08 | Epoch Time: 0m 34s
	Train Loss: 4.473 | Train Acc: 8.39%
	 Val. Loss: 4.491 |  Val. Acc: 6.68%
Epoch: 09 | Epoch Time: 0m 34s
	Train Loss: 4.447 | Train Acc: 9.51%
	 Val. Loss: 4.468 |  Val. Acc: 6.65%
Epoch: 10 | Epoch Time: 0m 34s
	Trai

KeyboardInterrupt: ignored